<a href="https://colab.research.google.com/github/mvdheram/AHCVS_Notes/blob/main/10_CoherentHierarchicalCulling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Coherent hierarchical culling

Link : https://onlinelibrary.wiley.com/doi/epdf/10.1111/j.1467-8659.2004.00793.x

* **Why ?**
  * Graphics hardware (GPU) support occlusion query using z-buffer.
  * There is a delay between issuing the query and actual processing in the graphics pipeline. 
  * **CHC - Algorithm used to optimize occlusion culling queries in complex scenes**
* **What ?**
  * **Algorithm to optimize usage of hardware occlusion culling queries in complex scenes**
* **How ?**
  * **Minimize the issued queries and reduces delays due to latency of result (CPU stalls, GPU starvation)**.
    * Scheduling of queris using spatial and temporal coherence of visibility.
    * Reuse the results of occlusion queries from the last frame in order to initiate and schedule the queries in the next frame.

**Questions:**

* Work done by Rendering pipeline 
  * GPU ?
    * Visivility query (culling), geometry transformation, Rasterization, 
  * CPU ?
    * Application (OpenGL,..) used for triangulation of the 3D scene
    * Two steps of disburdening graphics pipeline
      * Approximation , visibility culling 

## Why Background ?


* Hierarchical z-buffer algorithm
  * 

* **Main problem**
  * Latency betweeen the query and availability result
    * CPU sends the visible nodes of data structure to GPU for rendering 
      * As visibility culling is done at the GPU, CPU has to waite befor processing the next node in DS (Starvation)
   * latency causes two problems:   
    1. **CPU stalls : Waiting for the visibility query result of the node from GPU** 
    2. **GPU starvation : GPU waiting for the next node to be rendered.**  

## CHC Algo

**Solution** : **multiple occlusion queries for independent scene parts**
 

CHC exploits Temporal coherence by

1. **Initiate occlusion query** on the **nodes of hierarchy** where **traversal terminated in the laste frame**
  * Avoid queries on previously visible interior nodes

2. Assume previously visible leaf node is visible and render geometry without waiting

3. All the occlusion queries are store in query queue until the GPU result is known.


### Algorithm overview

1. Traverse the kd-tree in **front to back order**; skip Opened nodes and traverse termination nodes 
  * Apply occlusion query for termination nodes
    * **What?** 
      * At each frame of the tree traversal, there are
        * **Termination nodes (Ti)**
          * Group of nodes at depth d of the tree where the tree traversal has been terminated for the laste frame.
          * Consists of 
            * leaf nodes
            * Invisible nodes 
        * **Opened nodes (Oi)**
          * Interior nodes that have been classified as visible in the last frame.

2. Issue occlusion query for termination nodes 
  * For Leaf node
    * Visible  
      * Issue query and store in query queue
      * Render geometry without waiting
    * Invisible
      * Issue query and store in query queue
      * Dont render the node

3. Check Query queue
  * If query result  
    * Available
      * If node result == visible: 
          1. Process children recursively 
          2. Remove the visible node (becomes open node) and include the children to be processed into termination nodes.
            * Pull down the termination nodes (nodes at depth d of the kdtree) after traversal of the node 
      * If node result == Invisible
          * If node == inner node
              * Check if the siblings of the parent are invisible as well.
              * If true, the parent node becomes invible 
                * Pull up termination node ( remove the children from the termination node and add parent to the termination node) 

     * Not available
      * Continue recursively processing other nodes of kd tree     
  